In [17]:
import subprocess
import os

In [1]:
# read "archive_articles.json"
import json

# Read archive_articles.json
with open('archive_articles.json', 'r', encoding='utf-8') as file:
    archive_articles = json.load(file)

print(f"Loaded {len(archive_articles)} articles from archive_articles.json")






Loaded 26 articles from archive_articles.json


In [2]:
import logging


In [3]:
logging.basicConfig(level=logging.DEBUG)


In [4]:
from text_analysis.NEExtractor import NEExtractor

extractor = NEExtractor()


c:\Users\ra76lax\AppData\Local\anaconda3\envs\python_m3_scrapers\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\ra76lax\AppData\Local\anaconda3\envs\python_m3_scrapers\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\ra76lax\AppData\Local\anaconda3\envs\python_m3_scrapers\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
INFO:datasets:PyTorch version 2.2.2 available.
INFO:text_analysis.NEExtractor:Initializing NEExtractor with model: flair/ner-german-larg

2024-09-16 10:21:25,325 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


INFO:text_analysis.NEExtractor:Tagger instantiated successfully


In [5]:
archive_articles_inferred_gro_focus = extractor.extract_entities(archive_articles[0:3])


INFO:text_analysis.NEExtractor:Starting entity extraction for 3 articles
INFO:text_analysis.NEExtractor:Applying entity extraction to all articles


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

DEBUG:text_analysis.NEExtractor:Processing chunk 1 of 1
DEBUG:text_analysis.NEExtractor:Extracted 24 unique entities
DEBUG:text_analysis.NEExtractor:Processing chunk 1 of 1
DEBUG:text_analysis.NEExtractor:Extracted 21 unique entities
DEBUG:text_analysis.NEExtractor:Processing chunk 1 of 1
DEBUG:text_analysis.NEExtractor:Extracted 37 unique entities
INFO:text_analysis.NEExtractor:Inferring geographical focus
INFO:text_analysis.NEExtractor:Starting geographical focus inference for 3 articles
DEBUG:text_analysis.NEExtractor:Wrote main texts to file: main_texts_for_geo_focus_inference.txt
DEBUG:text_analysis.NEExtractor:Calling R script: text_analysis\infer_geo_focus.R
DEBUG:text_analysis.NEExtractor:Added geographical focus 'India' to article 0
DEBUG:text_analysis.NEExtractor:Added geographical focus 'Denmark' to article 1
DEBUG:text_analysis.NEExtractor:Added geographical focus 'Russia' to article 2
INFO:text_analysis.NEExtractor:Geographical focus inference completed successfully
INFO:t

In [6]:
archive_articles_inferred_gro_focus

[{'url': 'https://www.zeit.de/1946/10/der-kostenpunkt/komplettansicht',
  'main_text': 'Der Kostenpunkt – Seite 1 Achtzig Millionen Pfund Sterling sind im britischen Staatshaushalt für die Kosten der Besatzung von Deutschland vorgesehen, eine "verblüffende Zahl", wie der Sprecher der Opposition, Eden, bemerkte. Der Schatzkanzler Dalton, der den Staatshaushalt dem Parlament vorlegte, bezeichnete diese Summe als hoch und erklärte; daß vom britischen Steuerzahler nicht erwartet werden könne, daß er in dieser Höhe Zahlungen leiste, die Reparationen an Deutschland gleichkämen. Achtzig Millionen Pfund! In einer Zeit, die mit Milliardensummen zu rechnen sich gewöhnt hat, mag das ein kleiner Betrag sein. Großbritannien hat im abgelaufenen Etatsjahr nahezu fünfeinhalb Milliarden Pfund ausgegeben. Für das laufende Staatshaushaltsjahr sind Ausgaben von über 3,8 Milliarden Pfund vorgesehen, was zwar eine Herabsetzung von rund einem Drittel gegenüber der Kriegszeit bedeutet, aber immer noch einen g

In [2]:
!cd

c:\Users\ra76lax\Documents\GitHub\m3-scraper


In [16]:
#read archive_articles.json and write only the main_text to a txt file
with open('archive_articles.json', 'r') as f:
    data = json.load(f)
    with open('main_texts_for_geo_focus_inference.txt', 'w') as f:
        for item in data:
            f.write(item['main_text'] + '\n')


In [1]:
import subprocess
import json
import os
import base64

# Example data
articles = [
    {"main_text": "Some text about Germany", "central_entities": []},
    {"main_text": "Text about France", "central_entities": []}
]

# Encode the articles as a base64 string to avoid issues with special characters
articles_json = json.dumps(articles)
articles_base64 = base64.b64encode(articles_json.encode()).decode()

# Construct the R command
r_script_path = os.path.join('text_analysis', 'infer_geo_focus.R')
r_command = [
    'Rscript', 
    '-e', 
    f'''
    source("{r_script_path}")
    library(jsonlite)
    articles_json <- rawToChar(base64_dec("{articles_base64}"))
    articles <- fromJSON(articles_json)
    result <- infer_geo_focus_py(articles)
    cat(toJSON(result, auto_unbox=TRUE))
    '''
]

# Run the R script and capture the output
result = subprocess.run(r_command, capture_output=True, text=True, check=True)

# Parse the JSON output
country_names = json.loads(result.stdout)

print("Inferred country names:", country_names)

CalledProcessError: Command '['Rscript', '-e', '\n    source("text_analysis\\infer_geo_focus.R")\n    library(jsonlite)\n    articles_json <- rawToChar(base64_dec("W3sibWFpbl90ZXh0IjogIlNvbWUgdGV4dCBhYm91dCBHZXJtYW55IiwgImNlbnRyYWxfZW50aXRpZXMiOiBbXX0sIHsibWFpbl90ZXh0IjogIlRleHQgYWJvdXQgRnJhbmNlIiwgImNlbnRyYWxfZW50aXRpZXMiOiBbXX1d"))\n    articles <- fromJSON(articles_json)\n    result <- infer_geo_focus_py(articles)\n    cat(toJSON(result, auto_unbox=TRUE))\n    ']' returned non-zero exit status 1.

In [9]:
# Example data
articles = [
    {"main_text": "Some text about Germany", "central_entities": []},
    {"main_text": "Text about France", "central_entities": []}
]

In [10]:
import json

articles_json = json.dumps(articles).encode('utf-8')

In [11]:
articles_json

b'[{"main_text": "Some text about Germany", "central_entities": []}, {"main_text": "Text about France", "central_entities": []}]'

In [12]:
# Construct the R command
r_script_path = 'text_analysis/debug_json_input.R'
r_command = ['Rscript', r_script_path]


In [13]:
import subprocess

In [14]:
# Run the R script and capture the output
try:
    result = subprocess.run(r_command, input=articles_json, capture_output=True, text=True, check=True)
    
    print("R script stderr (for debugging):")
    print(result.stderr)

    print("\nR script stdout:")
    print(result.stdout)

except subprocess.CalledProcessError as e:
    print(f"R script failed with return code {e.returncode}")
    print("stderr:")
    print(e.stderr)
    print("stdout:")
    print(e.stdout)

TypeError: write() argument must be str, not bytes

In [13]:
# Parse the JSON output
try:
    parsed_output = json.loads(result.stdout)
    print("\nParsed output:")
    print(json.dumps(parsed_output, indent=2))
except json.JSONDecodeError as e:
    print(f"\nError parsing JSON output: {e}")
    print("Raw output:", result.stdout)

NameError: name 'result' is not defined

In [1]:
import subprocess
import json

def infer_geo_focus(articles_list):
    # Convert the list of dictionaries to JSON
    articles_json = json.dumps(articles_list)
    
    # Call the R script and pass the JSON as input
    result = subprocess.run(['Rscript', 'text_analysis/infer_geo_focus.R'], 
                            input=articles_json,
                            capture_output=True, 
                            text=True)
    
    print("R script return code:", result.returncode)
    print("R script stderr:")
    print(result.stderr)
    print("R script stdout:")
    print(result.stdout)
    
    if result.returncode != 0:
        raise Exception(f"R script failed: {result.stderr}")
    
    # Parse the output from the R script
    if not result.stdout.strip():
        print("R script produced no output")
        return articles_list
    
    try:
        geo_focus_results = json.loads(result.stdout)
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON output from R script: {e}")
        return articles_list
    
    # Add the geo focus results to the original articles
    for article, geo_focus in zip(articles_list, geo_focus_results):
        if geo_focus and geo_focus != 'NA':
            article['central_entities'].append({"type_id": "loc", "title": geo_focus})
    
    return articles_list

# Usage
articles = [
    {"main_text": "Some text about Germany", "central_entities": []},
    {"main_text": "Text about France", "central_entities": []}
]

try:
    result = infer_geo_focus(articles)
    print("Final result:")
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")

R script return code: 0
R script stderr:
Package version: 4.1.0
Unicode version: 15.1
ICU version: 74.1
Parallel computing: 8 of 8 threads used.
See https://quanteda.io for tutorials and examples.

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Error reading JSON input: lexical error: invalid char in json text.
                                       if (is.null(articles)) {  cat("
                     (right here) ------^

R script stdout:

R script produced no output
Final result:
[{'main_text': 'Some text about Germany', 'central_entities': []}, {'main_text': 'Text about France', 'central_entities': []}]
